In [1]:
import json
import re
import pandas as pd
import urllib
from tqdm import tqdm
from urllib.request import Request
from bs4 import BeautifulSoup
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def parseData(offer):
    dto = {}
    try:
        dto['name'] = offer['name']
    except KeyError:
        dto['name'] = None
    try:
        dto['voivodeship'] = offer['itemOffered']['address']['addressRegion']
    except KeyError:
        dto['voivodeship'] = None
    try:
        dto['city'] = offer['itemOffered']['address']['addressLocality']
    except KeyError:
        dto['city'] = None
    try:
        dto['price'] = offer['price']
    except KeyError:
        dto['price'] = None
    try:
        dto['currency'] = offer['currency']
    except KeyError:
        dto['currency'] = None
    try:
        dto['rooms'] = offer['itemOffered']['numberOfRooms']
    except KeyError:
        dto['rooms'] = None
    try:
        dto['size'] = offer['itemOffered']['floorSize']['value']
    except KeyError:
        dto['size'] = None
    return dto

In [3]:
apartaments = []

In [9]:
url = 'https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/cala-polska?page=1&limit=36&ownerTypeSingleSelect=ALL&by=DEFAULT&direction=DESC&viewType=listing&page='

for page_number in tqdm(range(3203,3787)):
    with urllib.request.urlopen(f'{url}{1}') as resp:
        page = BeautifulSoup(resp.read().decode('utf-8'), "html.parser")
        html = page.find(attrs={'type' : 'application/ld+json'})
        data = json.loads(html.text)
        for offer in data['@graph'][2]['offers']['offers']:
            apartaments.append(parseData(offer))

100%|██████████| 584/584 [09:22<00:00,  1.04it/s]


In [10]:
df = pd.DataFrame.from_records(apartaments)

In [12]:
df

,name,voivodeship,city,price,currency,rooms,size
0,Mieszkanie 64.10 m2 osiedle Południe (Jednostk...,kujawsko-pomorskie,Włocławek,355000.0,None,3,64.10
1,Do remontu miła okolica idealne pod inwestycje,lubelskie,Lublin,340000.0,None,2,49.00
2,"Mieszkanie na sprzedaż, Lublin, Czuby Południowe,",lubelskie,Lublin,480000.0,None,3,59.60
3,"Apartament wtulony w las, 4 miejsca parkingowe.",pomorskie,Gdynia,2700000.0,None,3,93.50
4,Mieszkanie 56m2-3pok/ul. Kamienna/Balkon..,dolnośląskie,Wrocław,459006.0,None,3,56.03
...,...,...,...,...,...,...,...
133216,"Przestronne, 4 pok., 2 poziomowe M4 na Krzykach.",dolnośląskie,Wrocław,899000.0,None,4,101.40
133217,"Pokoje- 65 mkw, 2 piwnice! Ustawne i jasne",dolnośląskie,Wrocław,724900.0,None,3,65.10
133218,Przy Grunwaldzie! Zrewitalizowana Kamienica!,dolnośląskie,Wrocław,799000.0,None,3,86.27
133219,>> taras 18m2 z widokiem na rzekę,dolnośląskie,Wrocław,799000.0,None,2,45.18


In [13]:
df.to_pickle('../Data/Otodom/otodom_09_05_23.zip')